# NYC Housing data

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
plt.style.use('ggplot')
%matplotlib inline

ImportError: No module named statsmodels.api

In [ ]:
file = 'rollingsales_manhattan.xls'
df = pd.read_excel(file, header=4)
df.dropna(inplace=True)
df.columns = [c.lower().replace(' ', '_') for c in df.columns]

In [ ]:
df.head()

### Looking at distributions in the data


In [ ]:
lower = 1e4
upper = 1e10
subset_df = df[(df["sale_price"] >= lower) & (df["sale_price"] <= upper) 
               & (df["gross_square_feet"] != 0) & (df["residential_units"] != 0)]

In [ ]:
plt.figure(figsize=(20,10))
x = pd.Categorical.from_array(subset_df["neighborhood"])
y = subset_df["sale_price"]
sns.boxplot(x.codes, y, orient="v")

In [ ]:
plt.figure(figsize=(20,10))
x = pd.Categorical.from_array(subset_df["building_class_at_time_of_sale"])
y = subset_df["sale_price"]
sns.boxplot(x.codes, y, orient="v")

In [ ]:
sns.regplot(subset_df["gross_square_feet"], subset_df["sale_price"])

In [ ]:
sns.regplot(subset_df["total_units"], subset_df["sale_price"])

### Linear Regression

In [ ]:
#formula = ('sale_price ~ C(neighborhood)')
#formula = ('sale_price ~ C(building_class_category)')
#formula = ('sale_price ~ gross_square_feet')
#formula = ('sale_price ~ total_units')
#formula = ('sale_price ~ gross_square_feet + total_units')
formula = ('sale_price ~ C(neighborhood) + C(building_class_at_time_of_sale) + gross_square_feet + total_units')
model = smf.ols(formula, subset_df).fit()
model.summary()

In [ ]:
sns.regplot(model.fittedvalues, model.resid)

### k-NN

In [ ]:
predictors = ['sale_price', 'total_units']
dfTrain, dfTest = train_test_split(subset_df)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(dfTrain[predictors], dfTrain['neighborhood'])
expected = dfTest['neighborhood']
predicted = knn.predict(dfTest[predictors])
error_rate = (predicted != expected).mean()
error_rate